In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240715'

In [3]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/5.engine_2'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['make_1.ABARTH.xlsx',
 'make_10.AMERICAN AUSTIN.xlsx',
 'make_100.GOLIATH.xlsx',
 'make_101.GORDON-KEEBLE.xlsx',
 'make_102.GRAHAM.xlsx',
 'make_103.GRAHAM-PAIGE.xlsx',
 'make_104.GRIFFITH.xlsx',
 'make_105.HEALEY.xlsx',
 'make_106.HENRY J.xlsx',
 'make_107.HILLMAN.xlsx',
 'make_108.HINO.xlsx',
 'make_109.HISPANO-SUIZA.xlsx',
 'make_11.AMERICAN BANTAM.xlsx',
 'make_110.HONDA.xlsx',
 'make_111.HOTCHKISS.xlsx',
 'make_112.HRG.xlsx',
 'make_113.HUDSON.xlsx',
 'make_114.HUMBER.xlsx',
 'make_115.HUMMER.xlsx',
 'make_116.HUPMOBILE.xlsx',
 'make_117.HYUNDAI.xlsx',
 'make_118.INFINITI.xlsx',
 'make_119.INTERNATIONAL.xlsx',
 'make_12.AMERICAN MOTORS.xlsx',
 'make_120.ISO.xlsx',
 'make_121.ISUZU.xlsx',
 'make_122.IVECO.xlsx',
 'make_123.JAC.xlsx',
 'make_124.JAGUAR.xlsx',
 'make_125.JEEP.xlsx',
 'make_126.JENSEN.xlsx',
 'make_127.JEWETT.xlsx',
 'make_128.JMC.xlsx',
 'make_129.JORDAN.xlsx',
 'make_13.AMPHICAR.xlsx',
 'make_130.JOWETT.xlsx',
 'make_131.KAISER-FRAZER.xlsx',
 'make_132.KARMA.xlsx',

In [4]:
for file in list_file:

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./{param_date}/5.engine_2/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['Make No'] = input_['Make No'].astype(int)
    input_['Model No'] = input_['Model No'].astype(int)
    input_['Engine No'] = input_['Engine No'].astype(int)
    input_['Year'] = input_['Year'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            import requests
            import json

            import sys
            sys.path.append('../../00.Tools')
            from crawler_configuration import get_header, get_proxy

            payload = {'jsn': {'tab': 'catalog',
                               'make': input_.loc[a, 'Make'],
                               'year': int(input_.loc[a, 'Year']),
                               'model': input_.loc[a, 'Model'],
                               'carcode': input_.loc[a, 'Car Code'],
                               'engine': input_.loc[a, 'Engine'],
                               'nodetype': 'carcode'}}

            data = {'func': 'navnode_fetch',
                    'payload': json.dumps(payload),
                    'api_json_request': '1'}

            try:
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                                 data=data,
                                                 headers=get_header(),
                                                 proxies=get_proxy(),
                                                 timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 200:
                        json_ = resp.json()
                        if 'redirect_to_url' in json_:
                            continue
                        elif json_['jsn']['tab'] == 'catalog' and json_['jsn']['make'] == input_.loc[a, 'Make'] and json_['jsn']['year'] == input_.loc[a, 'Year'] and json_['jsn']['model'] == input_.loc[a, 'Model'] and json_['jsn']['carcode'] == input_.loc[a, 'Car Code'] and json_['jsn']['engine'] == input_.loc[a, 'Engine'] and json_['jsn']['nodetype'] == 'carcode' and 'collected_javascript' in json_:
                            break

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_json = html.xpath('//input[contains(@id, "jsn[__GIP__")]/@value')

                # = = = = = = = = = = = = = = =

                list_group = []
                for json_ in list_json:
                    json_ = json.loads(json_)
                    if 'groupname' in json_ and json_['groupname'].strip() != '':
                        list_group.append(json_['groupname'])

                # = = = = = = = = = = = = = = =

                if not list_group:
                    raise

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                                        'Make': input_.loc[a, 'Make'],
                                        'Model No': input_.loc[a, 'Model No'],
                                        'Model': input_.loc[a, 'Model'],
                                        'Engine No': input_.loc[a, 'Engine No'],
                                        'Engine': input_.loc[a, 'Engine'],
                                        'Car Code': input_.loc[a, 'Car Code'],
                                        'Year': input_.loc[a, 'Year'],
                                        'Group No': [i+1 for i in range(len(list_group))],
                                        'Group': list_group})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                                         'Make': input_.loc[a, 'Make'],
                                         'Model No': input_.loc[a, 'Model No'],
                                         'Model': input_.loc[a, 'Model'],
                                         'Engine No': input_.loc[a, 'Engine No'],
                                         'Engine': input_.loc[a, 'Engine'],
                                         'Car Code': input_.loc[a, 'Car Code'],
                                         'Year': input_.loc[a, 'Year']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Make No']}.{input_.loc[a, 'Make']} > {input_.loc[a, 'Model No']}.{input_.loc[a, 'Model']} > {input_.loc[a, 'Engine No']}.{input_.loc[a, 'Engine']} > {input_.loc[a, 'Year']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(17):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Make No', 'Model No', 'Engine No', 'Year', 'Group No'],
                                                    ascending=[True, True, True, False, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./{param_date}/6.group_1/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Make No', 'Model No', 'Engine No', 'Year'],
                                                ascending=[True, True, True, False],
                                                ignore_index=True)
        output_error.to_excel(f'''./{param_date}/6.group_1/0.error/{file.removesuffix('.xlsx')}-group_1_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：ok，尝试次数：3] - 100.00% - make_99.GMC > 99.GMC > 192.YUKON XL 2500 > 3.8.1L V8 > 2006
[剩余数量：0] - [当前时间：18:40:14]

输出ing...

Done ~
